In [1]:
import mido
import numpy as np 
from matplotlib import pyplot as plt
import os
import time
import pickle

In [2]:
def midi_to_pianoRoll(data_folder):
    samples = []

    for dirpath, _, filenames in os.walk(data_folder):
        for File in filenames:
            path = os.path.join(dirpath, File)
            try:
                mid = mido.MidiFile(path)
                track = mido.merge_tracks(mid.tracks)
                abs_time = 0
                sample = []
                
                for msg in track:
                    abs_time += msg.time
                    
                    [sample.append([0]*128) for i in range(msg.time)]
                    if not msg.is_meta and msg.channel != 0:
                        continue

                    if msg.type == "note_off":
                        note = msg.note
                        for i in range(abs_time - msg.time,abs_time):
                            sample[i][note] = 1

                samples.append(sample)
            except:
                print(path)
                continue
    return samples

In [3]:
def getData(data_folder):
    """
    Walk through a folder, modifing each mid to fit in 128 ticks per beat and be only a piano channel
    :param data_folder: the base data folder used in the model, still nonrepresentative
    :return: it saves a midfile in piano format and other in paino+128tpb
    """
    counter = 0
    
    for dirpath, _, filenames in os.walk(data_folder):
        for File in filenames:
            path = os.path.join(dirpath, File)
            mid_copy = mido.MidiFile()
            
            try:
                mid = mido.MidiFile(path)
                tpb = mid.ticks_per_beat
                duration = int(mid.length)
                mid_copy.type = 1

                if 10 < duration < 300 and tpb in [48, 96, 120, 192, 240, 336, 384, 480, 960, 1024]:
                    track = mido.merge_tracks(mid.tracks)

                    for msg in list(track):
                        try:
                            if not msg.is_meta and msg.channel != 0 and msg.type == "note_on":
                                msg.velocity = 0
                        except:
                            continue

                    mid_copy.tracks.append(track)
                    mid_copy.ticks_per_beat = 128
                    mid_copy.save(f"OnlyPiano_TPB128/{File}") 
                    
            except:
                print(path)
                continue
                
            counter+=1
            if counter %250 ==0:
                print(counter)

In [4]:
start_time = time.time()
getData("TestDataset")
print("--- %s seconds ---" % (time.time() - start_time))

250
500
750
TestDataset\Balloon_Fight_-_Main_Theme_%28Dancing_Balloon%21_remix%29.mid
TestDataset\Battleofholy.mid
1000
1250
1500
1750
2000
2250
2500
2750
TestDataset\DarkMan2.mid
3000
TestDataset\DG_C3_Evergreen.mid
TestDataset\DG_C3_Mad_Forest.mid
TestDataset\DG_C3_Stream.mid
TestDataset\DG_C3_Vampire_Killer.mid
TestDataset\DG_MM3_Boss.mid
TestDataset\DG_MM3_Magnetman.mid
TestDataset\DG_MM5_Darkman.mid
TestDataset\DG_SMB_Main.mid
TestDataset\DG_SML_Muda.mid
TestDataset\Dive_Man_%28Cecil%27s_Extended_Dance_Mix%29.mid
3250
3500
3750
4000
4250
4500
4750
TestDataset\FF3_Crystal_Cave.mid
5000
5250
5500
5750
6000
6250
6500
6750
7000
TestDataset\KHmem_title.mid
7250
TestDataset\kidicboss1.mid
7500
7750
TestDataset\LosAngeles.mid
TestDataset\Low_HP_Alarm.mid
8000
8250
TestDataset\Maxhawk.mid
8500
8750
9000
TestDataset\MM3Pass.mid
9250
TestDataset\MM4_Dive_Man%28V1.1%29.mid
TestDataset\MM5GB_Jet_Stage_%28V1.1%29.mid
9500
9750
TestDataset\MM_Bomb_Man%28V1.1%29.mid
10000
10250
10500
TestDataset

In [1]:
# start_time = time.time()
# samples = midi_to_pianoRoll("OnlyPiano_TPB128")
# a_file = open("samples.pkl", "wb")
# pickle.dump(samples, a_file)
# a_file.close()
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
a_file = open("samples.pkl", "rb")
output = pickle.load(a_file)

In [ ]:

# def find_sample_range(samples):
#     """
#     Find the lowest and highest note in a sample
#     :param samples: List of samples
#     :return min_note, max_note: The equivalent lowest note in the param shapes and its max note
#     """
#     #Creates the lowest sample that can be played with the param's shapes
#     merged_sample = np.zeros_like(samples[0])

#     for sample in samples:
#         merged_sample = np.maximum(merged_sample, sample) #Maximum compare two arrays and returns element-wise maxima

#     merged_sample = np.amax(merged_sample, axis=0)
    
#     min_note = np.argmax(merged_sample) #Returns first of the maximum values along an axis
#     max_note = merged_sample.shape[0] - np.argmax(merged_sample[::-1]) #Returns last of the maximum values along an axis
    
#     return min_note, max_note

# def generate_centered_transpose(samples):
#     """
#     Center samples towards the middle of the pitch range (between the min_note and max_note of the sample).
#     :param samples: list of samples
#     :return out_samples: the original sample but centered based on min and max notes
#     :return out_lengths: the size of the song
#     """
#     num_notes = samples[0].shape[1]
#     min_note, max_note = find_sample_range(samples)

#     # find deviation from pitch center
#     center_deviation = num_notes/2 - (max_note + min_note)/2

#     out_samples = samples
#     out_lengths = [len(samples), len(samples)]

#     # center every sample by moving it by center_deviation
#     for i in range(len(samples)):
#         out_sample = np.zeros_like(samples[i])
#         out_sample[:, min_note+int(center_deviation) : max_note+int(center_deviation)] = samples[i][:, min_note:max_note]
#         out_samples.append(out_sample)

#     return out_samples, out_lengths

In [ ]:
# all_samples = []
# all_lens = []

# for mid in sig4f4_mids:
#     samples = midi_to_samples(mid, 16)
#     samples, lens = generate_centered_transpose(samples)
#     all_lens += lens
#     all_samples += samples

# all_samples = np.array(all_samples, dtype=np.uint8)
# all_lens = np.array(all_lens, dtype=np.uint32)
# np.save('samples424.npy', all_samples)
# np.save('lengths424.npy', all_lens)

## verificando que as musicas são iguais

In [ ]:
# for t in zip(teste[1].tracks, pianoRoll_mids[1].tracks):
#     for msg in zip(t[0],t[1]):
#         if msg[1] != msg[0]:
#             print(msg[1], msg[0])
#             print("")

# yield